In [1]:
import numpy as np
import os
import sys
import datetime

import scipy as sp
import scipy.stats

from bisect import bisect

from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector

import math





def time_now():#return time
    curr_time = datetime.datetime.now()
    return curr_time.strftime("%c")

def Convert_wig_into_bp_coverage(extracted_coverage,extracted_3UTR_region,strand_info):
    bp_coverage = np.zeros(extracted_3UTR_region[-1] - extracted_3UTR_region[0])
    relative_start = extracted_3UTR_region[0]
    for i in range(len(extracted_coverage)):
        curr_region_start = extracted_3UTR_region[i] - relative_start
        curr_region_end = extracted_3UTR_region[i+1] - relative_start
        bp_coverage[curr_region_start:curr_region_end] = extracted_coverage[i]
    if strand_info == '-':
        bp_coverage = bp_coverage[::-1]
    
    return bp_coverage
        
def parse_cfgfile(cfg_file):
    '''Parse configure file
    '''
    Group1_Tophat_aligned_file=''
    Group2_Tophat_aligned_file=''
    output_directory=''
    Annotated_3UTR_file=''
    Output_result_file=''
    Num_least_in_group1_local=''
    Num_least_in_group2_local=''
    Coverage_cutoff_local = ''
    FDR_cutoff_local = ''
    Fold_change_cutoff_local = ''
    PDUI_cutoff_local = ''
    APA_limit_file='' 

    for line in open(cfg_file,'r'):
        if line[0] == '\n' or line[0] == '#':
            comments = line;
        else:
            line = line.rstrip();
            command = line.split('=');
            if command[0] == 'Group1_Tophat_aligned_Wig':
                Group1_Tophat_aligned_file = command[1].split(',');
            if command[0] == 'Group2_Tophat_aligned_Wig':
                Group2_Tophat_aligned_file = command[1].split(',');
            if command[0] == 'APA_limit_file':
                APA_limit_file = command[1];
            if command[0] == 'Output_directory':
                output_directory = command[1]
                if output_directory[-1] != '/':
                    output_directory += '/'
            if command[0] == 'Annotated_3UTR':
                Annotated_3UTR_file = command[1]
            if command[0] == 'Output_result_file':
                Output_result_file = command[1]
            
            ##Parameters
            if command[0] == 'Num_least_in_group1':
                Num_least_in_group1_local = command[1]
            if command[0] == 'Num_least_in_group2':
                Num_least_in_group2_local = command[1]
            if command[0] == 'Coverage_cutoff':
                Coverage_cutoff_local = command[1]
            if command[0] == 'FDR_cutoff':
                FDR_cutoff_local = command[1]
            if command[0] == 'Fold_change_cutoff':
                Fold_change_cutoff_local = command[1]
            if command[0] == 'PDUI_cutoff':
                PDUI_cutoff_local = command[1]
            
    
    if Group1_Tophat_aligned_file=='':
        print >> sys.stderr, "No Tophat aligned BAM file for group 1!"
        exit(1)
    if Group2_Tophat_aligned_file=='':
        print >> sys.stderr, "No Tophat aligned BAM file for group 2!"
        exit(1)
    if APA_limit_file=='':
        print >> sys.stderr, "No APA limit file!"
        exit(1)
    if output_directory=='':
        print >> sys.stderr, "No output directory!"
        exit(1)
    if Annotated_3UTR_file=='':
        print >> sys.stderr, "No annotated 3' UTR file!"
        exit(1)
    if Output_result_file=='':
        print >> sys.stderr, "No result file name!"
        exit(1)
    return Group1_Tophat_aligned_file,Group2_Tophat_aligned_file,APA_limit_file,output_directory,Annotated_3UTR_file,Output_result_file,Num_least_in_group1_local,Num_least_in_group2_local,Coverage_cutoff_local,FDR_cutoff_local,Fold_change_cutoff_local,PDUI_cutoff_local

In [ ]:
def De_Novo_3UTR_Identification_Loading_Target_Wig_for_TCGA_Multiple_Samples_Main(argv=None):
    '''
    '''
    if len(sys.argv) == 1:
        print "Please provide the configure file ..."
        exit(1)

In [30]:
cfg_file = "/storage2/huanglu/report/APA/test/adv_dapars/data_configureSRR4496792VSSRR4496781.txt"
os.chdir("/storage2/huanglu/report/APA/test/adv_dapars")
print >> sys.stderr, "[%s] Start Analysis ..." % time_now()
Group1_Tophat_aligned_file,Group2_Tophat_aligned_file,APA_limit_file,output_directory,Annotated_3UTR_file,Output_result_file,Num_least_in_group1_local,Num_least_in_group2_local,Coverage_cutoff_local,FDR_cutoff_local,Fold_change_cutoff_local,PDUI_cutoff_local = parse_cfgfile(cfg_file)

num_group_1 = len(Group1_Tophat_aligned_file)
All_Sample_files = Group1_Tophat_aligned_file[:]
All_Sample_files.extend(Group2_Tophat_aligned_file)


global Num_least_in_group1
global Num_least_in_group2
global Coverage_cutoff
global FDR_cutoff
global Fold_change_cutoff
global PDUI_cutoff

if Num_least_in_group1_local != '':
    Num_least_in_group1 = float(Num_least_in_group1_local)
if Num_least_in_group2_local != '':
    Num_least_in_group2 = float(Num_least_in_group2_local)
if Coverage_cutoff_local != '':
    Coverage_cutoff = float(Coverage_cutoff_local)
if FDR_cutoff_local != '':
    FDR_cutoff = float(FDR_cutoff_local)
if Fold_change_cutoff_local != '':
    Fold_change_cutoff = float(Fold_change_cutoff_local)
if PDUI_cutoff_local != '':
    PDUI_cutoff = float(PDUI_cutoff_local)



##Prepare output directory
d = os.path.dirname(output_directory)
if not os.path.exists(d):
    os.makedirs(d)
temp_dir = d+'/tmp/'
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)
Output_all_prediction_file = output_directory+Output_result_file+'_result_temp.txt'
Output_result = open(Output_all_prediction_file, 'w')

num_samples = len(All_Sample_files)

##Debug
print >> sys.stderr, "[%s] Loading coverage ..." % time_now()
UTR_chr_set,APA_limit_dict=Load_Target_APA(APA_limit_file)
All_samples_Target_3UTR_coverages, All_samples_sequencing_depths, UTR_events_dict = Load_Target_Wig_files(All_Sample_files, Annotated_3UTR_file)
All_sample_coverage_weights = All_samples_sequencing_depths/np.mean(All_samples_sequencing_depths)
print >> sys.stderr, "[%s] Loading coverage finished ..." % time_now()
##Write the first line
first_line = ['Gene','fit_value','Predicted_Proximal_APA','Loci']
for i in range(num_group_1):
    curr_long_exp = 'A_%s_long_exp' % str(i+1)
    curr_short_exp = 'A_%s_short_exp' % str(i+1)
    curr_ratio ='A_%s_PDUI' % str(i+1)
    first_line.extend([curr_long_exp,curr_short_exp,curr_ratio])
for i in range(num_samples - num_group_1):
    curr_long_exp = 'B_%s_long_exp' % str(i+1)
    curr_short_exp = 'B_%s_short_exp' % str(i+1)
    curr_ratio ='B_%s_PDUI' % str(i+1)
    first_line.extend([curr_long_exp,curr_short_exp,curr_ratio])
first_line.append('PDUI_Group_diff')

Output_result.writelines('\t'.join(first_line) + '\n')

#print UTR_chr_set
#print APA_limit_dict
#print "hello"

for curr_3UTR_id in UTR_events_dict:
    curr_3UTR_structure = UTR_events_dict[curr_3UTR_id]
    curr_chrom = curr_3UTR_structure[0]
    region_start = curr_3UTR_structure[1]
    region_end   = curr_3UTR_structure[2]
    curr_strand  = curr_3UTR_structure[-2]
    UTR_pos = curr_3UTR_structure[-1]
    if curr_chrom in UTR_chr_set and curr_3UTR_id in All_samples_Target_3UTR_coverages:
        search_set=APA_limit_dict[curr_chrom]
        print curr_chrom,region_start,region_end
        print ':',
        #print search_set
        #print region_start,region_end
        limit_search_set=[x for x in search_set if( x <max(region_start,region_end)and x>min(region_start,region_end))]
        print limit_search_set
        if(len(limit_search_set)<=0):
            continue;
        curr_3UTR_coverage_wig = All_samples_Target_3UTR_coverages[curr_3UTR_id]
        curr_3UTR_all_samples_bp_coverage = []
        for curr_sample_curr_3UTR_coverage_wig in curr_3UTR_coverage_wig: 
            curr_3UTR_curr_sample_bp_coverage = Convert_wig_into_bp_coverage(curr_sample_curr_3UTR_coverage_wig[0],curr_sample_curr_3UTR_coverage_wig[1],curr_strand)
            curr_3UTR_all_samples_bp_coverage.append(curr_3UTR_curr_sample_bp_coverage)

        select_mean_squared_error_list,selcted_break_point_list,UTR_abundances_list = De_Novo_3UTR_Coverage_estimation_Genome_for_TCGA_multiple_samples(curr_3UTR_all_samples_bp_coverage, region_start, region_end,curr_strand,All_sample_coverage_weights,limit_search_set)

        if str(select_mean_squared_error_list) != "Na":
            for UTRlist in range(len(UTR_abundances_list)):
                select_mean_squared_error=select_mean_squared_error_list[UTRlist]
                selcted_break_point=selcted_break_point_list[UTRlist]
                UTR_abundances=UTR_abundances_list[UTRlist]
                Long_3UTR_exp_all = np.array(UTR_abundances[0])
                Short_3UTR_exp_all = np.array(UTR_abundances[1])
                num_non_zero = sum((Long_3UTR_exp_all + Short_3UTR_exp_all)>0)
                if num_non_zero == num_samples:
                    All_Long_inclusion_ratios = []
                    line_write = [curr_3UTR_id, "%.1f" % select_mean_squared_error, str(selcted_break_point), UTR_pos]
                    for i in range(num_samples):
                        curr_sample_ratio = float(UTR_abundances[0][i])/(float(UTR_abundances[0][i]) + float(UTR_abundances[1][i]))##long 3'UTR percentage
                        All_Long_inclusion_ratios.append(curr_sample_ratio)
                        line_write.append("%.2f" % UTR_abundances[0][i])
                        line_write.append("%.2f" % UTR_abundances[1][i])
                        line_write.append("%.2f" % curr_sample_ratio)

                    Group1_IR = All_Long_inclusion_ratios[:num_group_1]
                    Group2_IR = All_Long_inclusion_ratios[num_group_1:]
                    inclusion_ratio_Group_diff = np.mean(np.array(Group1_IR)) - np.mean(np.array(Group2_IR))

                    line_write.append("%.2f" % inclusion_ratio_Group_diff)

                    Output_result.writelines( '\t'.join(line_write)+'\n')

Output_result.close()

[Sat 04 Aug 2018 08:35:41 PM ] Start Analysis ...
[Sat 04 Aug 2018 08:35:41 PM ] Loading coverage ...
[Sat 04 Aug 2018 08:38:17 PM ] Loading coverage finished ...


chr11 33726024 33731888
: [33731649, 33730591, 33730664, 33727841, 33730850, 33731079, 33728819, 33729848, 33730046, 33730532, 33726830, 33731438, 33729751, 33731545, 33730418, 33731583, 33726428, 33730898, 33726535, 33728913, 33730983]
chr6 45879584 45880475
: []
chr7 120935495 120937096
: [120935662, 120936770, 120936253]
chr14 62215144 62215806
: []
chr11 59573748 59574304
: [59573981]
chr11 55889850 55890598
: []
chr2 27532459 27532848
: []
chr2 25551526 25551589
: []
chr15 64429768 64435099
: [64430201, 64433173, 64434636, 64434987]
chr3 197399022 197402066
: [197400679, 197400879]
chr1 212537997 212538717
: [212538304]
chr1 22987306 22987883
: []
chr11 33106272 33106834
: [33106520]
chr1 113454934 113456786
: [113455675, 113456305, 113456646, 113454982, 113456395, 113456116, 113455385, 113456143, 113455031, 113455774]
chr7 96649838 96650376
: []
chr19 6416442 6416521
: []
chr20 54978525 54979369
: [54979308, 54979053, 54979103, 54978975]
chr11 36483893 36486181
: [36484464, 36486

: []
chr16 48394635 48395387
: [48394966, 48395224, 48395344, 48394654, 48395263, 48395333]
chr2 75117925 75119969
: [75118147, 75118266]
chr1 52485981 52486683
: [52486332]
chr1 169822367 169822912
: [169822522, 169822401]
chr13 100214946 100215209
: [100215171, 100215061]
chr15 66794375 66794428
: []
chr3 115439642 115440194
: [115439924]
chr11 8959240 8959719
: []
chr10 69970044 69971426
: [69970839]
chr1 58946483 58946845
: [58946620]
chr9 91615970 91619133
: [91618967, 91618559]
chr19 29503419 29504572
: [29503773, 29503849]
chr17 80319757 80321272
: []
chr19 52022930 52023508
: [52022985]
chr12 49726364 49727015
: [49726414]
chr7 143955976 143956720
: []
chr6 70925952 70926783
: [70926228]
chr9 138476896 138478544
: []
chr3 195295661 195296005
: [195295855]
chr2 95956782 95956999
: []
chr19 17452288 17453288
: [17452833]
chr9 35737094 35737443
: []
chr1 153516159 153516398
: [153516245]
chr7 137074725 137076081
: [137075617]
chr1 12921077 12921625
: []
chr17 38175461 38175897
: [

: []
chr3 13545969 13547532
: [13546833, 13546884]
chr21 43407953 43411997
: [43408754, 43410528, 43410454, 43408341, 43410882, 43408146]
chr1 153579528 153580167
: []
chr5 71500912 71504499
: [71502549, 71501459, 71502881, 71502090, 71502419, 71502639, 71501696]
chr1 247198698 247201725
: [247198855, 247200088]
chr5 39372127 39373526
: [39373142, 39372226, 39373308]
chr1 162352973 162355880
: [162353768, 162353493]
chr13 41949542 41950840
: [41950836]
chr12 9208216 9208333
: []
chr10 7602002 7605076
: [7604692, 7603631, 7603034, 7604391, 7604742, 7602221, 7604483]
chr1 52811733 52812232
: [52811906, 52812204, 52811977]
chr2 42669670 42671718
: []
chr2 1546194 1546437
: [1546419]
chr19 15945313 15946045
: []
chr5 156184593 156184973
: [156184965]
chr3 47050710 47051096
: []
chr18 21111846 21112247
: [21112198, 21111934]
chr12 58010568 58010935
: []
chr10 120927382 120928044
: [120927643]
chr17 48459176 48460595
: [48459763, 48460037]
chr16 2326065 2326805
: [2326530]
chr4 4237413 42379

chr14 91924208 91925212
: [91925072, 91924356, 91924485]
chr11 102813997 102815094
: [102814489]
chr2 72406474 72406589
: []
chr12 56708357 56708749
: [56708530, 56708386, 56708578, 56708515]
chr6 132929580 132930440
: [132930345]
chr13 113832061 113832600
: [113832071]
chr16 30389532 30389840
: []
chr9 125281421 125282172
: []
chr10 14913528 14913696
: []
chr19 52249273 52250257
: []
chr10 124608680 124608954
: []
chr19 13991257 13991507
: []
chr7 127231268 127231660
: []
chr6 154431491 154431567
: []
chr12 97345728 97347120
: [97346526, 97346038, 97346876, 97346317, 97347001, 97345971, 97346425, 97346126]
chr7 149244690 149246464
: []
chr6 15520301 15521861
: [15521138, 15521779, 15520649]
chr9 131938374 131940539
: [131938386, 131938942]
chr4 7043007 7044727
: []
chr11 82972604 82973007
: [82972806, 82972671]
chr18 21534468 21534916
: [21534839]
chr4 154701976 154702906
: [154702293, 154702337]
chr3 46783680 46784072
: []
chr10 8056601 8056690
: []
chr22 19437626 19438266
: [1943784

...

: [100031295]
chr8 125500876 125500936
: []
chr17 18441993 18445500
: []
chr9 139377948 139380004
: []
chr4 74321349 74321462
: []
chr19 55512129 55512433
: []
chr21 36410532 36411722
: []
chr22 32545536 32545599
: []
chr12 10145708 10145885
: []
chr10 5149654 5149832
: []
chr17 3563531 3565655
: [3565304, 3564834, 3564756]
chr4 186321439 186321631
: [186321470]
chr6 29141312 29142142
: []
chr4 73940804 73942006
: [73941408, 73940877, 73941549, 73940950, 73941254]
chr17 28791607 28795660
: [28793875, 28794356, 28792050, 28794714, 28793950, 28791822, 28795424, 28793432, 28791970, 28794971, 28793269]
chr8 116422043 116427311
: [116426152, 116424534, 116422423, 116424583, 116424010, 116423783, 116424078, 116422607, 116423876]
chr3 155008145 155008634
: [155008557]
chr17 45900876 45901823
: [45901529]
chr15 50150883 50152671
: [50151260, 50151205, 50151904]
chr8 79513978 79516796
: [79515301, 79516577, 79514494, 79514144, 79515174, 79515368, 79516166, 79515446, 79514265, 79514579]
chr12 10

: [28239953, 28240395, 28240462]
chr16 24922650 24922884
: []
chr19 41633807 41634185
: []
chr1 40033074 40033181
: []
chr11 6496431 6498497
: [6497751, 6498249]
chr17 4534387 4535073
: []
chr17 7474692 7475167
: [7474942]
chr9 138418196 138418347
: []
chr19 5891691 5893300
: [5892950]
chr1 161254130 161255017
: []
chr17 37209119 37209389
: []
chr10 114186977 114187905
: [114187500, 114187311, 114187370, 114187033, 114187186]
chr2 233250720 233251753
: []
chr11 10533904 10536612
: [10534549, 10534604, 10535106, 10534815, 10536279, 10534381, 10535874]
chr8 30040592 30041041
: [30040983, 30040883]
chr12 2968830 2968934
: []
chr16 14766446 14766605
: []
chr2 166804546 166804773
: []
chr2 188343351 188343529
: []
chr5 131701205 131701290
: []
chr4 122745818 122747147
: [122745916]
chr19 10795536 10796260
: []
chr10 131978378 131978591
: []
chr3 186917340 186918869
: []
chr11 100226849 100229061
: []
chr11 121061416 121061494
: []
chr1 173205193 173209161
: []
chr21 39426466 39427074
: []
c

: [22418110]
chr5 52978949 52979125
: [52979102]
chr18 61170564 61171966
: [61171065]
chr9 131007244 131007308
: []
chr22 50920997 50924091
: [50922284]
chr13 52023959 52025309
: [52024363]
chr19 19040096 19040433
: []
chr11 74915446 74917044
: [74916747, 74915759]
chr19 48421687 48421782
: []
chr17 5377713 5378108
: []
chr10 99374683 99376167
: [99374795]
chr19 6940036 6940377
: []
chr12 56663935 56664586
: [56664476]
chr7 6501132 6502805
: [6501453, 6502687, 6502352, 6501638, 6502595, 6502504, 6502193, 6501393]
chr7 141920365 141920942
: []
chr6 143875648 143876366
: []
chr4 141471385 141474215
: [141471848, 141471496, 141472094]
chr6 29364417 29365241
: []
chr1 45119778 45120880
: [45119848, 45120178, 45120083]
chr21 34442451 34444271
: []
chr3 170714489 170715891
: [170714743, 170715215, 170715546, 170715292, 170714553, 170714909, 170715123, 170715483]
chr19 51675324 51676488
: []
chr4 9452087 9452208
: []
chr12 21418568 21422700
: [21422331, 21421828]
chr12 110205818 110207812
: [

: [134399889, 134399952]
chr10 44052159 44053618
: []
chr1 147380073 147381129
: []
chr7 25159311 25163467
: [25162033, 25162764, 25162447, 25163321, 25162844, 25162510]
chr15 41148969 41149675
: [41149603]
chr1 28087007 28088939
: [28087186, 28088113, 28087624]
chr21 40777925 40778537
: [40778458]
chr12 121458249 121458860
: []
chr21 32007584 32007740
: []
chr4 89617458 89619022
: [89618191, 89618145]
chr19 15490936 15491240
: []
chr3 8263149 8264405
: []
chr6 112429465 112430784
: [112430078, 112429962, 112430240, 112430340]
chr17 74266276 74267157
: [74266785, 74266839]
chr6 166823559 166826374
: [166825925, 166824615, 166824682]
chr19 38102415 38104545
: [38103083, 38102742, 38104530, 38102511, 38104284, 38102747]
chr12 54404874 54406210
: [54405166, 54405243]
chr3 50355372 50355969
: []
chr22 22311517 22311966
: []
chr19 48712122 48715231
: [48712910, 48714164, 48714258, 48714046, 48714979, 48713745, 48714386]
chr18 67508481 67514753
: [67508901, 67510030, 67508782, 67512278]
chr2

: [12692409, 12692845]
chr19 11660366 11660982
: []
chr11 45931350 45931862
: [45931728]
chr14 50584192 50585570
: [50584889, 50584524, 50584243]
chr9 139217955 139221778
: [139220812]
chr9 139252467 139253738
: []
chr11 67559544 67560763
: []
chr14 20978632 20979150
: []
chr17 16676812 16678010
: []
chr17 73754057 73754207
: []
chr14 101513659 101513719
: []
chr13 95226469 95227105
: []
chr6 119135182 119137456
: [119135498]
chr16 21413597 21414157
: []
chr12 13093725 13093782
: []
chr1 16360107 16360456
: []
chr7 107559640 107561241
: [107561196, 107560616, 107560186, 107559903, 107560408]
chr1 146767113 146767379
: []
chr14 65511407 65511470
: []
chr9 131909667 131910912
: []
chr11 36594842 36600015
: [36595552, 36596794, 36594995, 36598897]
chr1 11345702 11347932
: [11346413, 11347672, 11346338, 11346256, 11346585]
chr4 39122631 39126807
: [39125753, 39123827, 39125067, 39123755, 39124540, 39123300, 39123882, 39124464, 39123534, 39122843, 39123451]
chr11 56043116 56043898
: []
chr1

In [ ]:
print >> sys.stderr, "[%s] Filtering the result ..." % time_now()

Output_Motif_filtered_result_file = output_directory+Output_result_file+'_All_Prediction_Results.txt'
#UTR_APA_Result_filtering(Output_all_prediction_file,Genome_seq_fasta,Output_Motif_filtered_result_file)

DaPars_Filtering(Output_all_prediction_file, num_samples,num_group_1 ,Output_Motif_filtered_result_file)


try:
    os.remove(Output_all_prediction_file)
except OSError:
    pass

try:
    os.rmdir(temp_dir)
except OSError:
    pass



print >> sys.stderr, "[%s] Finished!" % time_now()

In [ ]:
#def DaPars_Filtering_debug():
def DaPars_Filtering(input_file, num_samples,num_group1 ,output_file):

In [32]:
input_file=Output_all_prediction_file
num_samples=num_samples
num_group1=num_group_1
output_file=output_directory+Output_result_file+'_All_Prediction_Results_final.txt'
output_write = open(output_file,'w')
num_line = 0

result_dict = {}
All_P_values = []
Selected_events_id = []
All_mean_abundance = []

for line in open(input_file,'r'):
    if num_line > 0:
        fields = line.strip('\n').split('\t')
        group1_coverages = np.zeros(2)
        group2_coverages = np.zeros(2)
        num_group1_pass = 0
        group1_PDUIs = 0
        for i in range(num_group1):
            curr_long = fields[4+i*3]
            curr_short = fields[5+i*3]
            if curr_long != 'NA':
                curr_long  = float(curr_long)
                curr_short = float(curr_short)
                if curr_long + curr_short >= Coverage_cutoff:
                    group1_PDUIs = group1_PDUIs + float(fields[6+i*3])
                    num_group1_pass += 1
                    group1_coverages[0] = group1_coverages[0] + curr_long  
                    group1_coverages[1] = group1_coverages[1] + curr_short
                else:
                    fields[4+i*3] = 'NA'
                    fields[5+i*3] = 'NA'
                    fields[6+i*3] = 'NA'


        num_group2_pass = 0
        group2_PDUIs = 0
        for i in range(num_samples - num_group1):
            curr_long = fields[4+(i+num_group1)*3]
            curr_short = fields[5+(i+num_group1)*3]
            if curr_long != 'NA':
                curr_long  = float(curr_long)
                curr_short = float(curr_short)
                if curr_long + curr_short >= Coverage_cutoff:
                    group2_PDUIs = group2_PDUIs + float(fields[6+(i+num_group1)*3])
                    num_group2_pass += 1
                    group2_coverages[0] = group2_coverages[0] + curr_long  
                    group2_coverages[1] = group2_coverages[1] + curr_short
                else:
                    fields[4+(i+num_group1)*3] = 'NA'
                    fields[5+(i+num_group1)*3] = 'NA'
                    fields[6+(i+num_group1)*3] = 'NA'



        if num_group1_pass >= Num_least_in_group1 and num_group2_pass >= Num_least_in_group2:
            Final_group_diff = str(group1_PDUIs/num_group1_pass - group2_PDUIs/num_group2_pass)

            All_mean_abundance.append([group1_PDUIs/num_group1_pass, group2_PDUIs/num_group2_pass])

            fields[-1] = str(Final_group_diff)
            ratio_val,P_val = sp.stats.fisher_exact([group1_coverages/num_group1_pass,group2_coverages/num_group2_pass])

            All_P_values.append(P_val)
            Selected_events_id.append(fields[0]+"_"+fields[2])
#             print Final_group_diff
            #print ratio_val
        else:
            fields[-1] = 'NA'


        result_dict[fields[0]+"_"+fields[2]]=fields

    else:
        first_line = line.strip('\n').split('\t')      

    num_line += 1
# print(len(Selected_events_id))

result_dict

##Filtering
curr_fields_1_vallist=[]
stats = importr('stats')
All_p_adjust = stats.p_adjust(FloatVector(All_P_values), method = 'BH')
first_line.insert(-1,'Group_A_Mean_PDUI')
first_line.insert(-1,'Group_B_Mean_PDUI')
first_line.extend(['P_val','adjusted.P_val','Pass_Filter'])
output_write.writelines('\t'.join(first_line)+'\n')
for curr_event_id in result_dict:
# for ele in result_dict:  ##在一个curr_event_id下有多个cut点被留下的情况下，Selected_events_id中的一个curr_event_id往往对应多个点，因此不应以原来的ID为索引，否则获得的p值会出错
    mean_PDUI_group1 = 'NA'
    mean_PDUI_group2 = 'NA'
    curr_P_val = 'NA'
    curr_FDR_val = 'NA'
    Pass_filter = 'N'
    curr_fields = result_dict[curr_event_id]
#     curr_event_id= ele[0]
    if curr_event_id in Selected_events_id:
        sel_ind = Selected_events_id.index(curr_event_id)
        curr_P_val = str(All_P_values[sel_ind])
        curr_FDR_val = str(All_p_adjust[sel_ind])

        mean_PDUI_group1 = All_mean_abundance[sel_ind][0]
        mean_PDUI_group2 = All_mean_abundance[sel_ind][1]
#         print(curr_FDR_val+"/"+curr_fields[-1])

#         curr_fields_1_vallist.append(curr_fields[-1])

# print(len(curr_fields_1_vallist))

#         if curr_FDR_val=='NA' or curr_fields[-1]=='NA':
#             Pass_filter = 'N'
        if float(curr_FDR_val) <= FDR_cutoff and abs(float(curr_fields[-1]))>=PDUI_cutoff and abs(math.log((mean_PDUI_group1+1e-5)/(mean_PDUI_group2+1e-5),2))>=Fold_change_cutoff:
#             Pass_filter = 'Y'

    curr_fields.insert(-1,str(mean_PDUI_group1))
    curr_fields.insert(-1,str(mean_PDUI_group2))
    curr_fields.append(curr_P_val)
    curr_fields.append(curr_FDR_val)
    curr_fields.append(Pass_filter)

    output_write.writelines('\t'.join(curr_fields) +'\n')


output_write.close()

In [2]:
 
def get_version():
    return "0.9.1"
    
    

def De_Novo_3UTR_Coverage_estimation_Genome_for_TCGA_multiple_samples(All_Samples_curr_3UTR_coverages, UTR_start, UTR_end,curr_strand,weight_for_second_coverage,limit_search_set):
    '''For UTR-APA new
       Load one chromosome by chromosome
       Just for TCGA data analysis. So no peak evenness checking
       Jan-17-2013
       2-28-2013
    '''
    coverage_threshold = 20
    search_point_start     = 200
    search_point_end       = int(abs((UTR_end - UTR_start))*0.1)
    num_samples = len(All_Samples_curr_3UTR_coverages)
    ##read coverage
    Region_Coverages = []
    Region_mean_Coverages = []
    Region_first_100_coverage_all_samples = []
    for i in range(num_samples):
        curr_Region_Coverage_raw = All_Samples_curr_3UTR_coverages[i]##strand is reversed in load
        curr_Region_Coverage = curr_Region_Coverage_raw/weight_for_second_coverage[i]
        Region_mean_Coverages.append(np.mean(curr_Region_Coverage_raw))
        Region_Coverages.append(curr_Region_Coverage)
        curr_first_100_coverage = np.mean(curr_Region_Coverage_raw[0:99])
        Region_first_100_coverage_all_samples.append(curr_first_100_coverage)
    if sum(np.array(Region_first_100_coverage_all_samples) >= coverage_threshold) >= num_samples and UTR_end - UTR_start >= 150:
        if curr_strand == "+":
            search_region = range(UTR_start+search_point_start, UTR_end-search_point_end+1)
            limit_search_set=[x for x in limit_search_set if x in range(UTR_start+search_point_start, UTR_end-search_point_end+1)]
        else:
            search_region = range(UTR_end - search_point_start, UTR_start+search_point_end-1, -1)
            limit_search_set=[x for x in limit_search_set if x in range(UTR_end - search_point_start, UTR_start+search_point_end-1, -1)]
        if len(limit_search_set)<=0:
        	return [],[],[]
        #print limit_search_set
        search_region_start = search_point_start
        search_region_end   = UTR_end - UTR_start - search_point_end
        Mean_squared_error_list  = []
        Estimated_3UTR_abundance_list = []
        if curr_strand == "+":
        	test_list=[x-UTR_start for x in limit_search_set]
        else:
        	test_list=[UTR_end-x for x in limit_search_set]
        #for curr_point in range(search_region_start, search_region_end+1):
        for curr_point in test_list:
            curr_search_point = curr_point
            All_samples_result = [[],[],[]]
            for curr_sample_region_coverage in Region_Coverages:
                Mean_Squared_error,Long_UTR_abun,Short_UTR_abun = Estimation_abundance(curr_sample_region_coverage, curr_search_point)
                All_samples_result[0].append(Mean_Squared_error)
                All_samples_result[1].append(Long_UTR_abun)
                All_samples_result[2].append(Short_UTR_abun)
            
            Mean_Squared_error = np.mean(np.array(All_samples_result[0]))
            Mean_squared_error_list.append(Mean_Squared_error)
            Estimated_3UTR_abundance_list.append([All_samples_result[1],All_samples_result[2]])



        if len(Mean_squared_error_list) > 0:
            #min_ele_index = Mean_squared_error_list.index(min(Mean_squared_error_list))
            
            select_mean_squared_error = Mean_squared_error_list
            UTR_abundances = Estimated_3UTR_abundance_list
            selcted_break_point = limit_search_set
            ###huanglu问题？是返回一个值还是返回一组值？
        else:
            return [],[],[]
        
    else:
        return [],[],[]
    
    return select_mean_squared_error,selcted_break_point,UTR_abundances

def Estimation_abundance(Region_Coverage, break_point):
    Long_UTR_abun  = np.mean(Region_Coverage[break_point:])
    Short_UTR_abun = np.mean(Region_Coverage[0:break_point] - Long_UTR_abun)
    if Short_UTR_abun < 0:
        Short_UTR_abun = 0
    Coverage_diff  = Region_Coverage[0:break_point] - Long_UTR_abun - Short_UTR_abun
    Coverage_diff= np.append(Coverage_diff, Region_Coverage[break_point:] - Long_UTR_abun)
    Mean_Squared_error = np.mean(Coverage_diff**2)
    
    return Mean_Squared_error,Long_UTR_abun,Short_UTR_abun


    
def Load_Target_APA(APA_file):
    UTR_chr_set=set()
    APA_limit_dict={}
    for line in open(APA_file,'r'):
        fields = line.strip('\n').split('\t')
        name = curr_chr = fields[0]
        posi =int(fields[1])
        #curr_strand=fields[3]
        UTR_chr_set.add(name)
        if name not in APA_limit_dict:
            APA_limit_dict[curr_chr] = []
        APA_limit_dict[curr_chr].append(posi)
    return UTR_chr_set, APA_limit_dict

def Load_Target_Wig_files(All_Wig_files, UTR_Annotation_file):
    UTR_events_dict = {}
    All_Samples_Total_depth = []
    for line in open(UTR_Annotation_file,'r'):
        fields = line.strip('\n').split('\t')
        curr_chr = fields[0]
        region_start = fields[1]
        region_end   = fields[2]
        curr_strand  = fields[-1]
        UTR_pos = "%s:%s-%s" % (curr_chr, region_start, region_end)
        end_shift = int(round(abs(int(region_start) - int(region_end)) * 0.2))
        if curr_strand == '+':
            region_end = str(int(region_end) - end_shift)
        else:
            region_start = str(int(region_start) + end_shift)
        region_start = int(region_start) + 1
        region_end   = int(region_end) - 1
        if region_start + 50 < region_end:
            UTR_events_dict[fields[3]] = [fields[0],region_start,region_end,fields[-1],UTR_pos]

    ##Load coverage for all samples
    All_samples_extracted_3UTR_coverage_dict = {}
    for curr_wig_file in All_Wig_files:
        curr_sample_All_chroms_coverage_dict = {}
        num_line = 0
        cur_sample_total_depth = 0
        for line in open(curr_wig_file,'r'):
            if '#' not in line and line[0:3] == 'chr':
                fields = line.strip('\n').split('\t')
                chrom_name = fields[0]
                region_start = int(fields[1])
                region_end = int(fields[2])
                cur_sample_total_depth += int(float(fields[-1])) * (region_end - region_start)
                if chrom_name not in curr_sample_All_chroms_coverage_dict:
                    curr_sample_All_chroms_coverage_dict[chrom_name] = [[0],[0]]
                if region_start > curr_sample_All_chroms_coverage_dict[chrom_name][0][-1]:
                    curr_sample_All_chroms_coverage_dict[chrom_name][0].append(region_start)
                    curr_sample_All_chroms_coverage_dict[chrom_name][1].append(0)
                curr_sample_All_chroms_coverage_dict[chrom_name][0].append(region_end)
                curr_sample_All_chroms_coverage_dict[chrom_name][1].append(int(float(fields[-1])))
            num_line += 1
        curr_sample_All_chroms_coverage_dict[chrom_name][1].append(0)
        All_Samples_Total_depth.append(cur_sample_total_depth)
        for curr_3UTR_event_id in UTR_events_dict:
            curr_3UTR_structure = UTR_events_dict[curr_3UTR_event_id]
            curr_chr = curr_3UTR_structure[0]
            if curr_chr in curr_sample_All_chroms_coverage_dict:
                curr_chr_coverage = curr_sample_All_chroms_coverage_dict[curr_chr]
                region_start = curr_3UTR_structure[1]
                region_end = curr_3UTR_structure[2]
                left_region_index = bisect(curr_chr_coverage[0],region_start)
                right_region_index = bisect(curr_chr_coverage[0],region_end)

                extracted_coverage = curr_chr_coverage[1][left_region_index:right_region_index+1]
                extracted_3UTR_region = curr_chr_coverage[0][left_region_index:right_region_index]
                extracted_3UTR_region.insert(0,region_start)
                extracted_3UTR_region.append(region_end)
                if curr_3UTR_event_id not in All_samples_extracted_3UTR_coverage_dict:
                    All_samples_extracted_3UTR_coverage_dict[curr_3UTR_event_id] = []
                All_samples_extracted_3UTR_coverage_dict[curr_3UTR_event_id].append([extracted_coverage,extracted_3UTR_region])
    return All_samples_extracted_3UTR_coverage_dict,np.array(All_Samples_Total_depth),UTR_events_dict

In [ ]:
##Default Parameters
Num_least_in_group1 = 1
Num_least_in_group2 = 1
Coverage_cutoff = 30

FDR_cutoff = 0.05
PDUI_cutoff = 0.2
Fold_change_cutoff = 0.59 #1.5 fold change





if __name__ == '__main__':
    '''Non-paralle version for matched tumor-normal 3'UTR Identification.
       5-26-2013
    '''
    De_Novo_3UTR_Identification_Loading_Target_Wig_for_TCGA_Multiple_Samples_Main(sys.argv)
    
    ##debug
    #DaPars_Filtering_debug()
    

In [20]:
str1 = "this isexam string example....wow!!!exam";
str2 = "exam";
 
print str1.index(str2);

7
